<a href="https://colab.research.google.com/github/AutoRoboCulture/mask-rcnn-for-multiple-objects/blob/master/Mask_RCNN_for_multiple_objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 : Mount Drive + Clone MRCNN Lib + Load Dataset

MRCNN Code: [How to Train an Object Detection Model with Keras](https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/)

In [0]:
from google.colab import drive
drive.mount('/content/drive') 

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [0]:
%cd Mask_RCNN/

In [0]:
!python setup.py install

In [0]:
!pip show mask-rcnn

In [0]:
%pwd

In [0]:
#Upload your dataset and place it to /content/Mask_RCNN/

In [0]:
#Making preTrainedWeights directory
!mkdir /content/Mask_RCNN/weed_cfg_preTrainedWeights/

In [0]:
#Download preTrainedWeights from https://drive.google.com/open?id=1bXEOmOsoBLpXQWVvhhJvhD-RjvCLxOAQ

#**Step 2**: Train Model Here 
*Change load weight path before run


In [0]:
# fit a mask rcnn on the weed dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import pdb
import numpy as np
import skimage

#pdb.set_trace()

# class that defines and loads the weed dataset
class WeedDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define two class
		self.add_class("dataset", 1, "crop")
		self.add_class("dataset", 2, "weed")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			#print('IMAGE ID: ',image_id)
			# skip bad images
			if (image_id == '.ipynb_checkpo'):
				continue
			# skip all images after 115 if we are building the train set
			if is_train and int(image_id) >= 115:
				continue
			# skip all images before 115 if we are building the test/val set
			if not is_train and int(image_id) < 115:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		#for box in root.findall('.//bndbox'):
		for box in root.findall('.//object'):
			name = box.find('name').text
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			#coors = [xmin, ymin, xmax, ymax, name]
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		#pdb.set_trace()
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'crop'):
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('crop'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('weed'))
			
				
		return masks, asarray(class_ids, dtype='int32')

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define a configuration for the model
class WeedConfig(Config):
	# define the name of the configuration
	NAME = "weed_cfg"
	# number of classes (background + weed + crop)
	NUM_CLASSES = 1 + 2
	# number of training steps per epoch
	STEPS_PER_EPOCH = 115


# prepare train set
train_set = WeedDataset()
train_set.load_dataset('weed', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = WeedDataset()
test_set.load_dataset('weed', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = WeedConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights and exclude the output layers
model.load_weights('<enter your trained model path>', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=60, layers='heads')

In [0]:
#Backup weights to DRIVE

#STEP3: FOR Calculating testMAP
**write your preTrained Model path**

In [0]:
# fit a mask rcnn on the weed dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import pdb
import numpy as np
import skimage
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

# class that defines and loads the weed dataset
class WeedDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define two class
		self.add_class("dataset", 1, "crop")
		self.add_class("dataset", 2, "weed")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			#print('IMAGE ID: ',image_id)
			# skip bad images
			if (image_id == '.ipynb_checkpo'):
				continue
			# skip all images after 115 if we are building the train set
			if is_train and int(image_id) >= 115:
				continue
			# skip all images before 115 if we are building the test/val set
			if not is_train and int(image_id) < 115:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids = [0,1,2])

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		#for box in root.findall('.//bndbox'):
		for box in root.findall('.//object'):
			name = box.find('name').text
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			#coors = [xmin, ymin, xmax, ymax, name]
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		#pdb.set_trace()
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'crop'):
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('crop'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('weed'))
		return masks, asarray(class_ids, dtype='int32')
	
		# calculate the mAP for a model on a given dataset
		def evaluate_model(dataset, model, cfg):
		APs = list()
		for image_id in dataset.image_ids:
			# load image, bounding boxes and masks for the image id
			image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
			# convert pixel values (e.g. center)
			scaled_image = mold_image(image, cfg)
			# convert image into one sample
			sample = expand_dims(scaled_image, 0)
			# make prediction
			yhat = model.detect(sample, verbose=0)
			# extract results for first sample
			r = yhat[0]
			# calculate statistics, including AP
			AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
			# store
			APs.append(AP)
		# calculate the mean AP across all images
		mAP = mean(APs)
		return mAP

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define a configuration for the model
class WeedConfig(Config):
	# define the name of the configuration
	NAME = "weed_cfg"
	# number of classes (background + weed + crop)
	NUM_CLASSES = 1 + 2
	# number of training steps per epoch
	STEPS_PER_EPOCH = 115


# prepare train set
train_set = WeedDataset()
train_set.load_dataset('weed', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = WeedDataset()
test_set.load_dataset('weed', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = WeedConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights and exclude the output layers
model.load_weights('<enter your trained model path>', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)


# Step 4: For predict Image using PreTrained Weights (.h5)

In [0]:
# evaluate the mask rcnn model on the kangaroo dataset
import time
start1 = time.time()
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import skimage
import numpy as np
import pdb
import cv2

import glob
import pdb
import os
import re

# class that defines and loads the weed dataset
class WeedDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define classes
		self.add_class("dataset", 1, "crop")
		self.add_class("dataset", 2, "weed")
		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			# skip bad images
			if (image_id == '.ipynb_checkpo'):
				continue
			# skip all images after 115 if we are building the train set
			if is_train and int(image_id) >= 115:
				continue
			# skip all images before 115 if we are building the test/val set
			if not is_train and int(image_id) < 115:
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# get the root of the document
		root = tree.getroot()
		# extract each bounding box
		boxes = list()
		#for box in root.findall('.//bndbox'):
		for box in root.findall('.//object'):
			name = box.find('name').text
			xmin = int(box.find('./bndbox/xmin').text)
			ymin = int(box.find('./bndbox/ymin').text)
			xmax = int(box.find('./bndbox/xmax').text)
			ymax = int(box.find('./bndbox/ymax').text)
			#coors = [xmin, ymin, xmax, ymax, name]
			coors = [xmin, ymin, xmax, ymax, name]
			boxes.append(coors)
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	# load the masks for an image
	def load_mask(self, image_id):
		#pdb.set_trace()
		# get details of image
		info = self.image_info[image_id]
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if (box[4] == 'crop'):
				masks[row_s:row_e, col_s:col_e, i] = 2
				class_ids.append(self.class_names.index('crop'))
			else:
				masks[row_s:row_e, col_s:col_e, i] = 1
				class_ids.append(self.class_names.index('weed'))
		return masks, asarray(class_ids, dtype='int32')

	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "weed_cfg"
	# number of classes (background + weed + crop)
	NUM_CLASSES = 1 + 2
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=1):
	times = []
	imgExt = 'png'
	input_path = '<enter input image folder path/*.>' + imgExt
	output_path = '<enter output image save folder path/'
	nameR = input_path.replace('*.'+imgExt,'')
	pathR, dirsR, filesR = next(os.walk(nameR))
	file_count = len(filesR)
	#pdb.set_trace()

	img_num = 0     #file number initial
	
	cnt = 0
	del_img = []
	file_name = '%05d.'+imgExt
	output_path = output_path+file_name     #Output Image Filename
	#pdb.set_trace()
	sort_temp = []
	def natural_sort(l): 
		convert = lambda text: int(text) if text.isdigit() else text.lower() 
		alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
		return sorted(l, key = alphanum_key)

	for img_jpg1 in glob.glob(input_path):
			sort_temp.append(img_jpg1)
	sort = natural_sort(sort_temp)

	for img_jpg in sort:

			filename = output_path%int(img_num)        #Output Image filename
			cnt+=1
			img_num+=1
			print ('Renamed: ',cnt,'Out of: ',file_count)

			image = skimage.io.imread(img_jpg) #dataset.load_image(i+15)
			overlay = image.copy()
			sample = expand_dims(image, 0)
			start3 = time.time()
			yhat = model.detect(sample, verbose=0)[0]
			delta = (time.time() - start3)
			times.append(delta)
			boxCount = 0
			alpha = 0.4
			for classId in yhat['class_ids']:
				if (classId == 2):
					box = yhat['rois'][boxCount]
					# get coordinates
					y1, x1, y2, x2 = box
					cv2.rectangle(overlay, (x1,y1), (x2,y2), (150,0,0) , -1)
					predictedImage = cv2.addWeighted(overlay, alpha, image, 1-alpha, 0)
					predictedImage = cv2.cvtColor(predictedImage,cv2.COLOR_BGR2RGB)
				else:
					box = yhat['rois'][boxCount]
					# get coordinates
					y1, x1, y2, x2 = box
					cv2.rectangle(overlay, (x1,y1), (x2,y2), (0,0,150) , -1)
					predictedImage = cv2.addWeighted(overlay, alpha, image, 1-alpha, 0)
					predictedImage = cv2.cvtColor(predictedImage,cv2.COLOR_BGR2RGB)
				cv2.imwrite(filename,predictedImage)
				boxCount+=1

# load the train dataset
train_set = WeedDataset()
train_set.load_dataset('weed', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = WeedDataset()
test_set.load_dataset('weed', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model_path = '<enter your model pretrained weights path>'
model.load_weights(model_path, by_name=True)
# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)


In [0]:
#Backup weights to DRIVE